In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_val_score


%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
dft = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
dfs = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
df.shape

In [ ]:
dft.shape

In [ ]:
df.columns

In [ ]:
dft.columns

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
X=df.loc[:,'feature_0':'feature_49']

In [ ]:
y=df.loc[:,'target']

In [ ]:
y=np.ravel(y)

In [ ]:
X.describe()

###### Train

In [ ]:
for col in df.columns:
    print(df[col].value_counts())

###### Test

In [ ]:
for col in dft.columns:
    print(dft[col].value_counts())

In [ ]:
cor_matrix=df.corr(method="pearson")
cor_matrix

In [ ]:
for i in X.columns:
    sns.histplot(X[i])
    plt.show()

In [ ]:
label=LabelEncoder()
y=label.fit_transform(y)

In [ ]:
y

# TRAIN AND TEST SPLIT

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 50)

In [ ]:
cv=StratifiedKFold(n_splits=5)

# CREATING TWO PIPELINES FOR LOGISTIC AND RANDOM FOREST

In [ ]:
estimator_1=[( 'Scalar1',StandardScaler()),('pcal1',PCA(n_components=30)),
             ('cls1', RandomForestClassifier())] #created the input for pipelines

pipe_1= Pipeline(estimator_1) #first pipe line created


In [ ]:
pipe_1.fit(X_train, y_train)
p1_cv_scores = cross_val_score(pipe_1, X_train, y_train, cv=cv)

print('Pipe_1_train_Score,{},testScore,{}'.format(pipe_1.score(X_train, y_train),pipe_1.score(X_test, y_test)))
print('Pipe_1_cvmean,{},pipe_1_cvstd,{}'.format(np.mean(p1_cv_scores), np.std(p1_cv_scores)))

In [ ]:
pca=PCA()
logistic = LogisticRegression(max_iter=10000, tol=0.1)

logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe_2 = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

In [ ]:
pipe_2.fit(X_train, y_train)
p2_cv_scores = cross_val_score(pipe_2, X_train, y_train, cv=cv)

print('Pipe_2_train_Score,{},testScore,{}'.format(pipe_2.score(X_train, y_train),pipe_2.score(X_test, y_test)))
print('Pipe_2_cvmean,{},pipe_2_cvstd,{}'.format(np.mean(p2_cv_scores), np.std(p2_cv_scores)))

# GRID SEARCH _PARAMETER OPTIMIZATION

In [ ]:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe_2, param_grid, n_jobs=-1)
search.fit(X_train, y_train)

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# PREDICTION

In [ ]:
X_pred=dft.loc[:,'feature_0':'feature_49']

In [ ]:
predict = pipe_2.predict(X_pred)
print(predict)

In [ ]:
probability_predict = pipe_2.predict_proba(X_pred)
probability_predict

In [ ]:
dfs.iloc[:,1:5] = probability_predict
dfs

In [ ]:
dfs.to_csv('submission_log_2.csv',index=False)

In [ ]:
dfs.to_csv('tps_submission_log.csv',index=False)